In [1]:
import stripe
import pandas as pd
import numpy as np
import psycopg2
from time import sleep
from config import api_key
from config import host, database, user, password
from queries import *



In [2]:
def get_df():
    sub_df = pd.DataFrame()
    stripe.api_key = api_key
                                                        
    #first query to get response id
    first_response = stripe.Subscription.list(status='all')
    #creating dataframe with responses
    sub_df = sub_df.append(pd.json_normalize(first_response['data']))
    #defining next page for loops
    start_after = first_response['data'][-1]['id']
    print('Getting Results')

    while True:    
        try:
            #appending to the dataframe
            response = stripe.Subscription.list(starting_after=start_after, status='all')
            subscription_df = pd.json_normalize(response['data'])
            sub_df = sub_df.append(subscription_df, ignore_index=True)
            #giving the key for the next iteration
            start_after = response['data'][-1]['id']
            #ask george about rate limit issues
            sleep(1)
        except:
            print('Stripe Request went wrong')
        
        
        if response["has_more"] == False:
            break
    return sub_df


In [3]:
sub_df = get_df()

Getting Results


In [4]:
def cleaning_df(df):
    df = df[['id', 'billing_cycle_anchor', 'cancel_at', 'cancel_at_period_end', 'canceled_at', 'created', 'current_period_end', 'current_period_start', 'customer', 'ended_at', 'quantity', 'start_date', 'status']]
    df['billing_cycle_anchor'] = pd.to_datetime(sub_df['billing_cycle_anchor'], unit='s')
    df['canceled_at'] = pd.to_datetime(sub_df['canceled_at'], unit='s')
    df['cancel_at'] = pd.to_datetime(sub_df['cancel_at'], unit='s')
    df['created'] = pd.to_datetime(sub_df['created'], unit='s')
    df['current_period_end'] = pd.to_datetime(sub_df['current_period_end'], unit='s')
    df['current_period_start'] = pd.to_datetime(sub_df['current_period_start'], unit='s')
    df['ended_at'] = pd.to_datetime(sub_df['ended_at'], unit='s')
    df['start_date'] = pd.to_datetime(sub_df['start_date'], unit='s')
    df = df.replace({np.NaN: None})
    return df


In [5]:
sub_df = cleaning_df(sub_df)
sub_df

C:\Users\JOHNST~1\AppData\Local\Temp/ipykernel_28700/823688802.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['billing_cycle_anchor'] = pd.to_datetime(sub_df['billing_cycle_anchor'], unit='s')
C:\Users\JOHNST~1\AppData\Local\Temp/ipykernel_28700/823688802.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['canceled_at'] = pd.to_datetime(sub_df['canceled_at'], unit='s')
C:\Users\JOHNST~1\AppData\Local\Temp/ipykernel_28700/823688802.py:5: SettingWithCopyWarning: 
A value is trying to be set on a c

,id,billing_cycle_anchor,cancel_at,cancel_at_period_end,canceled_at,created,current_period_end,current_period_start,customer,ended_at,quantity,start_date,status
0,sub_1L3nGALnLiSMHjeFX3ocagbc,2022-05-26 20:26:18,None,False,None,2022-05-26 20:26:18,2022-06-26 20:26:18,2022-05-26 20:26:18,cus_LlJp4k1sEEQEkC,None,1,2022-05-26 20:26:18,active
1,sub_1L3luTLnLiSMHjeFXSCGprAu,2022-05-26 18:59:49,None,False,None,2022-05-26 18:59:49,2022-06-26 18:59:49,2022-05-26 18:59:49,cus_LZf1KNHRjry6hi,None,1,2022-05-26 18:59:49,active
2,sub_1L3XMPLnLiSMHjeFj07DXhXL,2022-05-26 03:27:41,None,False,None,2022-05-26 03:27:41,2022-06-26 03:27:41,2022-05-26 03:27:41,cus_KvBk0XN9n9Lvrh,None,1,2022-05-26 03:27:41,active
3,sub_1L3WoaLnLiSMHjeFs3tKzn7m,2022-05-26 02:52:44,None,False,None,2022-05-26 02:52:44,2022-06-26 02:52:44,2022-05-26 02:52:44,cus_JuM1h73O5htVm5,None,1,2022-05-26 02:52:44,active
4,sub_1L3WJ9LnLiSMHjeFrRmfaiUS,2022-05-26 02:20:15,None,False,None,2022-05-26 02:20:15,2022-06-26 02:20:15,2022-05-26 02:20:15,cus_LOubSJcQX5HSBS,None,1,2022-05-26 02:20:15,active
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,sub_HxRsSFzSCdOCao,2020-09-04 04:57:19,None,False,2020-12-21 20:49:31,2020-09-04 04:57:19,2021-01-04 04:57:19,2020-12-04 04:57:19,cus_HsxJ4mZ2mIvuaw,2020-12-21 20:49:31,1,2020-09-04 04:57:19,canceled
1190,sub_Hx4l1OOH4jFpCg,2020-09-03 05:03:49,None,False,2020-11-23 22:57:12,2020-09-03 05:03:49,2020-12-03 05:03:49,2020-11-03 05:03:49,cus_HsRzIRAfM5a8Pt,2020-11-23 22:57:12,1,2020-09-03 05:03:49,canceled
1191,sub_Hx4hMQdjs9NK8q,2020-09-03 05:00:03,None,False,2020-09-03 05:02:56,2020-09-03 05:00:03,2020-10-03 05:00:03,2020-09-03 05:00:03,cus_HsRzIRAfM5a8Pt,2020-09-03 05:02:56,1,2020-09-03 05:00:03,canceled
1192,sub_Hww73rW1qJJn9n,2020-09-02 20:07:40,None,False,2020-09-02 21:36:40,2020-09-02 20:07:40,2020-10-02 20:07:40,2020-09-02 20:07:40,cus_HsRzIRAfM5a8Pt,2020-09-02 21:36:40,1,2020-09-02 20:07:40,canceled


In [6]:
def connect(host, database, user, password):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(host=host, database=database, user=user, password=password)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
conn = connect(host, database, user, password)

Connecting to the PostgreSQL database...
Connection successful


In [7]:
def setup(conn, schema, table):
    cur = conn.cursor()
    cur.execute(schema)
    cur.execute(table)
    

In [8]:
setup(conn, create_subscription_schema, create_subscription_table)

In [9]:
def insert_values(conn, df, table):
    
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    cur = conn.cursor()
    values = [cur.mogrify("(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", tup).decode('utf8') for tup in tuples]
    query  = (f'INSERT INTO {table(cols)} VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT({values[0]}) DO UPDATE SET')
    
    try:
         cur.execute(query, tuples)
         conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cur.close()
        return 1
    print("execute_mogrify() done")
    cursor.close()
   

In [10]:
#insert_values(conn, sub_df, 'subscriptions')

In [11]:
sub_df.columns

Index(['id', 'billing_cycle_anchor', 'cancel_at', 'cancel_at_period_end',
       'canceled_at', 'created', 'current_period_end', 'current_period_start',
       'customer', 'ended_at', 'quantity', 'start_date', 'status'],
      dtype='object')

In [12]:
def insert_values_v2(conn, df):
    
    cur = conn.cursor()
    
    for value in df.values:
        subscription_id, billing_cycle_anchor, cancel_at, cancel_at_period_end, canceled_at, created, current_period_end, current_period_start, customer, ended_at, quantity, start_date, status = value
        print(value)
        #insert subscriber record
        subscriber_data = (subscription_id, billing_cycle_anchor, cancel_at, cancel_at_period_end, canceled_at, created, current_period_end, current_period_start, customer, ended_at,quantity, start_date, status)
        cur.execute("""
        INSERT INTO subscriptions VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
            ON CONFLICT (subscription_id)
            DO UPDATE SET
            subscription_id = EXCLUDED.subscription_id,
            billing_cycle_anchor = EXCLUDED.billing_cycle_anchor,
            cancel_at = EXCLUDED.cancel_at,
            cancel_at_period_end = EXCLUDED.cancel_at_period_end,
            canceled_at = EXCLUDED.canceled_at,
            created = EXCLUDED.created,
            current_period_end = EXCLUDED.current_period_end,
            current_period_start = EXCLUDED.current_period_start,
            customer_id = EXCLUDED.customer_id,
            ended_at = EXCLUDED.ended_at,
            quantity = EXCLUDED.quantity,
            start_date = EXCLUDED.start_date,
            status = EXCLUDED.status;
            """, subscriber_data)



In [13]:
insert_values_v2(conn, sub_df)

['sub_1L3nGALnLiSMHjeFX3ocagbc' Timestamp('2022-05-26 20:26:18') None
 False None Timestamp('2022-05-26 20:26:18')
 Timestamp('2022-06-26 20:26:18') Timestamp('2022-05-26 20:26:18')
 'cus_LlJp4k1sEEQEkC' None 1 Timestamp('2022-05-26 20:26:18') 'active']
['sub_1L3luTLnLiSMHjeFXSCGprAu' Timestamp('2022-05-26 18:59:49') None
 False None Timestamp('2022-05-26 18:59:49')
 Timestamp('2022-06-26 18:59:49') Timestamp('2022-05-26 18:59:49')
 'cus_LZf1KNHRjry6hi' None 1 Timestamp('2022-05-26 18:59:49') 'active']
['sub_1L3XMPLnLiSMHjeFj07DXhXL' Timestamp('2022-05-26 03:27:41') None
 False None Timestamp('2022-05-26 03:27:41')
 Timestamp('2022-06-26 03:27:41') Timestamp('2022-05-26 03:27:41')
 'cus_KvBk0XN9n9Lvrh' None 1 Timestamp('2022-05-26 03:27:41') 'active']
['sub_1L3WoaLnLiSMHjeFs3tKzn7m' Timestamp('2022-05-26 02:52:44') None
 False None Timestamp('2022-05-26 02:52:44')
 Timestamp('2022-06-26 02:52:44') Timestamp('2022-05-26 02:52:44')
 'cus_JuM1h73O5htVm5' None 1 Timestamp('2022-05-26 02:52

['sub_1KvWc9LnLiSMHjeFwL8NyHP0' Timestamp('2022-05-04 01:02:49') None
 False None Timestamp('2022-05-04 01:02:49')
 Timestamp('2022-06-04 01:02:49') Timestamp('2022-05-04 01:02:49')
 'cus_LEUR28eCSVHIvh' None 1 Timestamp('2022-05-04 01:02:49') 'active']
['sub_1KvVElLnLiSMHjeFcUWDgVJ0' Timestamp('2022-05-03 23:34:35') None
 False None Timestamp('2022-05-03 23:34:35')
 Timestamp('2022-06-03 23:34:35') Timestamp('2022-05-03 23:34:35')
 'cus_KnPApo1mT6fh7x' None 1 Timestamp('2022-05-03 23:34:35') 'active']
['sub_1KvUW3LnLiSMHjeFMdCvj82h' Timestamp('2022-05-03 22:48:22') None
 False None Timestamp('2022-05-03 22:48:22')
 Timestamp('2022-06-03 22:48:22') Timestamp('2022-05-03 22:48:22')
 'cus_LLvLnrImKpSpUs' None 1 Timestamp('2022-05-03 22:48:22') 'active']
['sub_1KvOWOLnLiSMHjeFJxvVMMDN' Timestamp('2022-05-03 16:24:20')
 Timestamp('2022-06-03 16:24:20') True Timestamp('2022-05-14 18:44:08')
 Timestamp('2022-05-03 16:24:20') Timestamp('2022-06-03 16:24:20')
 Timestamp('2022-05-03 16:24:20') 

['sub_1KhswPLnLiSMHjeFHDftPpGp' Timestamp('2022-03-27 10:03:21') None
 False Timestamp('2022-03-28 09:04:03') Timestamp('2022-03-27 10:03:21')
 Timestamp('2022-04-27 10:03:21') Timestamp('2022-03-27 10:03:21')
 'cus_L1wxS4YJol4zrS' Timestamp('2022-03-28 09:04:03') 1
 Timestamp('2022-03-27 10:03:21') 'incomplete_expired']
['sub_1Khp55LnLiSMHjeF0oeHd05Z' Timestamp('2022-03-27 05:56:03') None
 False None Timestamp('2022-03-27 05:56:03')
 Timestamp('2022-05-27 05:56:03') Timestamp('2022-04-27 05:56:03')
 'cus_LOc7Su7Td0IhlJ' None 1 Timestamp('2022-03-27 05:56:03') 'active']
['sub_1KhkFGLnLiSMHjeFNbAOy0Mh' Timestamp('2022-03-27 00:46:14') None
 False None Timestamp('2022-03-27 00:46:14')
 Timestamp('2022-05-27 00:46:14') Timestamp('2022-04-27 00:46:14')
 'cus_Jtrf24Y56cjYTD' None 1 Timestamp('2022-03-27 00:46:14') 'active']
['sub_1KhidHLnLiSMHjeF2b4Z2Jo9' Timestamp('2022-03-26 23:02:55') None
 False None Timestamp('2022-03-26 23:02:55')
 Timestamp('2022-06-26 23:02:55') Timestamp('2022-05-2

['sub_1KWXAWLnLiSMHjeFzfIJLWW5' Timestamp('2022-02-24 02:35:00') None
 False None Timestamp('2022-02-24 02:35:00')
 Timestamp('2022-06-24 02:35:00') Timestamp('2022-05-24 02:35:00')
 'cus_JuM1h73O5htVm5' None 1 Timestamp('2022-02-24 02:35:00') 'past_due']
['sub_1KWVY9LnLiSMHjeF10jtdUzW' Timestamp('2022-02-24 00:51:17') None
 False None Timestamp('2022-02-24 00:51:17')
 Timestamp('2022-06-24 00:51:17') Timestamp('2022-05-24 00:51:17')
 'cus_L0wFRCTpc339by' None 1 Timestamp('2022-02-24 00:51:17') 'active']
['sub_1KWSbTLnLiSMHjeFKQ4BYdAq' Timestamp('2022-02-23 21:42:31') None
 False Timestamp('2022-04-12 16:54:53') Timestamp('2022-02-23 21:42:31')
 Timestamp('2022-04-23 21:42:31') Timestamp('2022-03-23 21:42:31')
 'cus_LC6Mg0XXe9waYR' Timestamp('2022-04-12 16:54:53') 1
 Timestamp('2022-02-23 21:42:31') 'canceled']
['sub_1KWA6bLnLiSMHjeFVLKivCDx' Timestamp('2022-02-23 01:57:25') None
 False None Timestamp('2022-02-23 01:57:25')
 Timestamp('2022-06-23 01:57:25') Timestamp('2022-05-23 01:57:

['sub_1KGru7LnLiSMHjeFRNNbBlfa' Timestamp('2022-01-11 21:29:19')
 Timestamp('2022-02-11 21:29:19') True Timestamp('2022-01-11 21:37:35')
 Timestamp('2022-01-11 21:29:19') Timestamp('2022-02-11 21:29:19')
 Timestamp('2022-01-11 21:29:19') 'cus_KwlL7xW12Jnycs'
 Timestamp('2022-02-11 21:29:19') 1 Timestamp('2022-01-11 21:29:19')
 'canceled']
['sub_1KGd9cLnLiSMHjeFpkFfoTKC' Timestamp('2022-01-11 05:44:20') None
 False None Timestamp('2022-01-11 05:44:20')
 Timestamp('2022-06-11 05:44:20') Timestamp('2022-05-11 05:44:20')
 'cus_KwWAMt5BYOmYnu' None 1 Timestamp('2022-01-11 05:44:20') 'active']
['sub_1KGDckLnLiSMHjeFdCvp3sEi' Timestamp('2022-01-10 02:28:42') None
 False None Timestamp('2022-01-10 02:28:42')
 Timestamp('2022-06-10 02:28:42') Timestamp('2022-05-10 02:28:42')
 'cus_Kw5ZBGhG2dAcbv' None 1 Timestamp('2022-01-10 02:28:42') 'active']
['sub_1KG9RyLnLiSMHjeFd6t5c4ut' Timestamp('2022-01-09 22:01:18')
 Timestamp('2022-02-09 22:01:18') True Timestamp('2022-01-11 00:22:08')
 Timestamp('20

['sub_1JyJpZLnLiSMHjeFW6TmkOrO' Timestamp('2021-11-21 17:27:57')
 Timestamp('2021-12-21 17:27:57') True Timestamp('2021-11-21 17:30:19')
 Timestamp('2021-11-21 17:27:57') Timestamp('2021-12-21 17:27:57')
 Timestamp('2021-11-21 17:27:57') 'cus_KdayNwbqG7fstN'
 Timestamp('2021-12-21 17:27:57') 1 Timestamp('2021-11-21 17:27:57')
 'canceled']
['sub_1Jy1MeLnLiSMHjeFzX9QZtk2' Timestamp('2021-11-20 21:44:52') None
 False None Timestamp('2021-11-20 21:44:52')
 Timestamp('2022-06-20 21:44:52') Timestamp('2022-05-20 21:44:52')
 'cus_KdHtKDAmkPYDOT' None 1 Timestamp('2021-11-20 21:44:52') 'active']
['sub_1JxM2oLnLiSMHjeFrhVdMNpQ' Timestamp('2021-11-19 01:37:38') None
 False None Timestamp('2021-11-19 01:37:38')
 Timestamp('2022-06-19 01:37:38') Timestamp('2022-05-19 01:37:38')
 'cus_KcbCfs4zj3UTgp' None 1 Timestamp('2021-11-19 01:37:38') 'active']
['sub_1JxK50LnLiSMHjeFtToFTx2Z' Timestamp('2021-11-18 23:31:46') None
 False Timestamp('2022-01-03 16:16:13') Timestamp('2021-11-18 23:31:46')
 Timesta

['sub_1JboMjLnLiSMHjeFjjNGcbgy' Timestamp('2021-09-20 15:25:09') None
 False Timestamp('2021-09-20 20:54:09') Timestamp('2021-09-20 15:25:09')
 Timestamp('2021-10-20 15:25:09') Timestamp('2021-09-20 15:25:09')
 'cus_J669iyQqpHMRBB' Timestamp('2021-09-20 20:54:09') 1
 Timestamp('2021-09-20 15:25:09') 'incomplete_expired']
['sub_1JboMZLnLiSMHjeFEc3oMp3u' Timestamp('2021-09-20 15:24:59') None
 False Timestamp('2021-09-20 20:54:13') Timestamp('2021-09-20 15:24:59')
 Timestamp('2021-10-20 15:24:59') Timestamp('2021-09-20 15:24:59')
 'cus_J669iyQqpHMRBB' Timestamp('2021-09-20 20:54:13') 1
 Timestamp('2021-09-20 15:24:59') 'incomplete_expired']
['sub_1JboLwLnLiSMHjeFYhRjtqPm' Timestamp('2021-09-20 15:24:20') None
 False Timestamp('2021-09-20 20:54:17') Timestamp('2021-09-20 15:24:20')
 Timestamp('2021-10-20 15:24:20') Timestamp('2021-09-20 15:24:20')
 'cus_J669iyQqpHMRBB' Timestamp('2021-09-20 20:54:17') 1
 Timestamp('2021-09-20 15:24:20') 'incomplete_expired']
['sub_1JbbWzLnLiSMHjeFjfOWPrCY'

['sub_JzWbNGlLtEdxKd' Timestamp('2021-08-06 18:14:10')
 Timestamp('2021-09-06 18:14:10') True Timestamp('2021-08-08 00:57:52')
 Timestamp('2021-08-06 18:14:10') Timestamp('2021-09-06 18:14:10')
 Timestamp('2021-08-06 18:14:10') 'cus_JzWXsZ4XPuL7r0'
 Timestamp('2021-09-06 18:14:10') 1 Timestamp('2021-08-06 18:14:10')
 'canceled']
['sub_JzOmpIZhydgjzf' Timestamp('2021-08-06 10:09:16')
 Timestamp('2021-09-06 10:09:16') True Timestamp('2021-08-12 03:19:44')
 Timestamp('2021-08-06 10:09:16') Timestamp('2021-09-06 10:09:16')
 Timestamp('2021-08-06 10:09:16') 'cus_JNqbVXGzq4EZux'
 Timestamp('2021-09-06 10:09:16') 1 Timestamp('2021-08-06 10:09:16')
 'canceled']
['sub_JzJVAZbH0oPzwW' Timestamp('2021-08-06 04:42:02') None False None
 Timestamp('2021-08-06 04:42:02') Timestamp('2022-06-06 04:42:02')
 Timestamp('2022-05-06 04:42:02') 'cus_JzJUsKGb8nIZ1c' None 1
 Timestamp('2021-08-06 04:42:02') 'active']
['sub_JzHVBsSUk3oIz3' Timestamp('2021-08-06 02:38:05') None False
 Timestamp('2021-09-15 17:48

['sub_JQrqL5TkiEzsHJ' Timestamp('2021-05-06 05:51:03') None False
 Timestamp('2021-05-07 04:51:09') Timestamp('2021-05-06 05:51:03')
 Timestamp('2021-06-06 05:51:03') Timestamp('2021-05-06 05:51:03')
 'cus_JQrl5LV7fca6wF' Timestamp('2021-05-07 04:51:09') 1
 Timestamp('2021-05-06 05:51:03') 'incomplete_expired']
['sub_JQTJ2ua2bDrPGC' Timestamp('2021-05-05 04:30:19') None False None
 Timestamp('2021-05-05 04:30:19') Timestamp('2022-06-05 04:30:19')
 Timestamp('2022-05-05 04:30:19') 'cus_JQTIovVNOEYWUh' None 1
 Timestamp('2021-05-05 04:30:19') 'active']
['sub_JQP5IG0tUcpOdU' Timestamp('2021-05-05 00:07:41') None False None
 Timestamp('2021-05-05 00:07:41') Timestamp('2022-06-05 00:07:41')
 Timestamp('2022-05-05 00:07:41') 'cus_JCzbe1kuqJnk4y' None 1
 Timestamp('2021-05-05 00:07:41') 'active']
['sub_JPZt63NnYwc0YZ' Timestamp('2021-05-02 19:14:11') None False None
 Timestamp('2021-05-02 19:14:11') Timestamp('2022-06-02 19:14:11')
 Timestamp('2022-05-02 19:14:11') 'cus_JPZGTSbbtLWKg3' None 1

In [14]:
cur = conn.cursor()
sql = '''SELECT * FROM subscriptions'''
  
  
cur.execute(sql)
column_names = [desc[0] for desc in cur.description]
for i in column_names:
    print(i)

subscription_id
billing_cycle_anchor
cancel_at
cancel_at_period_end
canceled_at
created
current_period_end
current_period_start
customer_id
ended_at
quantity
start_date
status


In [15]:
sub_df

,id,billing_cycle_anchor,cancel_at,cancel_at_period_end,canceled_at,created,current_period_end,current_period_start,customer,ended_at,quantity,start_date,status
0,sub_1L3nGALnLiSMHjeFX3ocagbc,2022-05-26 20:26:18,None,False,None,2022-05-26 20:26:18,2022-06-26 20:26:18,2022-05-26 20:26:18,cus_LlJp4k1sEEQEkC,None,1,2022-05-26 20:26:18,active
1,sub_1L3luTLnLiSMHjeFXSCGprAu,2022-05-26 18:59:49,None,False,None,2022-05-26 18:59:49,2022-06-26 18:59:49,2022-05-26 18:59:49,cus_LZf1KNHRjry6hi,None,1,2022-05-26 18:59:49,active
2,sub_1L3XMPLnLiSMHjeFj07DXhXL,2022-05-26 03:27:41,None,False,None,2022-05-26 03:27:41,2022-06-26 03:27:41,2022-05-26 03:27:41,cus_KvBk0XN9n9Lvrh,None,1,2022-05-26 03:27:41,active
3,sub_1L3WoaLnLiSMHjeFs3tKzn7m,2022-05-26 02:52:44,None,False,None,2022-05-26 02:52:44,2022-06-26 02:52:44,2022-05-26 02:52:44,cus_JuM1h73O5htVm5,None,1,2022-05-26 02:52:44,active
4,sub_1L3WJ9LnLiSMHjeFrRmfaiUS,2022-05-26 02:20:15,None,False,None,2022-05-26 02:20:15,2022-06-26 02:20:15,2022-05-26 02:20:15,cus_LOubSJcQX5HSBS,None,1,2022-05-26 02:20:15,active
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,sub_HxRsSFzSCdOCao,2020-09-04 04:57:19,None,False,2020-12-21 20:49:31,2020-09-04 04:57:19,2021-01-04 04:57:19,2020-12-04 04:57:19,cus_HsxJ4mZ2mIvuaw,2020-12-21 20:49:31,1,2020-09-04 04:57:19,canceled
1190,sub_Hx4l1OOH4jFpCg,2020-09-03 05:03:49,None,False,2020-11-23 22:57:12,2020-09-03 05:03:49,2020-12-03 05:03:49,2020-11-03 05:03:49,cus_HsRzIRAfM5a8Pt,2020-11-23 22:57:12,1,2020-09-03 05:03:49,canceled
1191,sub_Hx4hMQdjs9NK8q,2020-09-03 05:00:03,None,False,2020-09-03 05:02:56,2020-09-03 05:00:03,2020-10-03 05:00:03,2020-09-03 05:00:03,cus_HsRzIRAfM5a8Pt,2020-09-03 05:02:56,1,2020-09-03 05:00:03,canceled
1192,sub_Hww73rW1qJJn9n,2020-09-02 20:07:40,None,False,2020-09-02 21:36:40,2020-09-02 20:07:40,2020-10-02 20:07:40,2020-09-02 20:07:40,cus_HsRzIRAfM5a8Pt,2020-09-02 21:36:40,1,2020-09-02 20:07:40,canceled


In [19]:
cur.execute('SELECT * FROM subscriptions')

In [18]:
print(cur.fetchall())

[('sub_1L3nGALnLiSMHjeFX3ocagbc', datetime.date(2022, 5, 26), None, False, None, datetime.date(2022, 5, 26), datetime.date(2022, 6, 26), datetime.date(2022, 5, 26), 'cus_LlJp4k1sEEQEkC', None, 1, datetime.date(2022, 5, 26), 'active'), ('sub_1L3luTLnLiSMHjeFXSCGprAu', datetime.date(2022, 5, 26), None, False, None, datetime.date(2022, 5, 26), datetime.date(2022, 6, 26), datetime.date(2022, 5, 26), 'cus_LZf1KNHRjry6hi', None, 1, datetime.date(2022, 5, 26), 'active'), ('sub_1L3XMPLnLiSMHjeFj07DXhXL', datetime.date(2022, 5, 26), None, False, None, datetime.date(2022, 5, 26), datetime.date(2022, 6, 26), datetime.date(2022, 5, 26), 'cus_KvBk0XN9n9Lvrh', None, 1, datetime.date(2022, 5, 26), 'active'), ('sub_1L3WoaLnLiSMHjeFs3tKzn7m', datetime.date(2022, 5, 26), None, False, None, datetime.date(2022, 5, 26), datetime.date(2022, 6, 26), datetime.date(2022, 5, 26), 'cus_JuM1h73O5htVm5', None, 1, datetime.date(2022, 5, 26), 'active'), ('sub_1L3WJ9LnLiSMHjeFrRmfaiUS', datetime.date(2022, 5, 26), No